In [2]:
# A IMPORTER

import pandas as pd
import json
import unidecode
import re
import numpy as np

## Cleaning the data

On charge notre dataset. Puis on créé un autre dataset reprenant les features qui nous intéresse pour appliquer nos processus NLP.

In [3]:
data = pd.read_json('indeed6.json', encoding = 'utf-8', lines=True)

In [6]:
job_to_class = data[['titre', 'texte']]

Ici, nous avons décidé d'utiliser la libraire spaCy. C'est elle qui nous permettra de parseriser nos descriptions et nos titres, retirer les stop words, et tokenizer.

In [7]:
import spacy 
#Chargeons notre vocabulaire spaCy
nlp_fr = spacy.load("fr_core_news_sm")

In [74]:
#Chargeons nos stopwords qu'on mettra dans une liste
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.fr.stop_words import STOP_WORDS
french_stopwords = list(spacy.lang.fr.stop_words.STOP_WORDS)
english_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
stop_words = french_stopwords + english_stopwords

#J'importe la ponctuation qui nous servira au traitement du texte
import string
punctuations = string.punctuation

#Ainsi que notre Parser qui comprend la structure grammatical de notre texte
from spacy.lang.fr import French
parser = French()

Dans un premier temps, on avait opté pour utiliser strictement spaCy pour le traitement global. Toutefois, il restait imparfait avec beaucoup de "déchet" dans nos tokens.
On choisit donc de subdiviser à nouveau notre traitement. 
Tout d'abord on retire les caractères indésirables. Puis, dans la mesure où on a déjà créé des nouvelles features grâce au regex, on supprime ces éléments de nos données afin d'éviter tout risque de colinéarité dans l'entraînement de nos modèles de machine learning.
Enfin on tokenize nos textes et on nettoie à nouveau les caractères indésirables qui ont pu se glisser dedans.

In [234]:
from string import digits

#Ces caractères indésirables ne nous fournissent pas d'information pertinente. On les retire
def clean_and_lower(string):
    forbid_car= [":", ";", ",", "&", "(", ")",
                '"', "!", "?", "*", "-", "\n", 
                "...", "/","'"]
    for car in forbid_car:
            cleaned_string = string.replace(car, ' ')
    return cleaned_string

#On retire les termes déjà étudié lors de nos regex, et qui sont déjà des features
def terme_a_retirer(string):
        string = re.sub(pattern = '(jeune[es().]* diplôme[es().]*|junior[s]*|novice[s]*| débutant[es().]*)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(expérimenté[es().]*|senior[s]*|confirmé[es().]*)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(licence|bac[ +]*[23/]+|iut|dut|bts| bsc|master|bac[ +]*5|ingénieur[.()es]*|grande[s]* école[s]*| msc|doctorat[s]*|docteur[.()es]*|ph[.]*d|thèse[s]*|bac)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(minimum [1-9]*[/aou-]*\d+ an[nées]* d\'expérience|[1-9]*[/aou-]*\d+ an[nées]* d\'expérience|[1-9]*[/aou-]*\d+ an[nées]* minimum d\'expérience|expérience minimum de *[1-9]*[/aou-]*\d+ an[nees]|expérience|d\'expérience)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(nantes|bordeaux|paris|île-de-france|lyon|toulouse)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '[\d+ ]*[ \-k€]*[\d+ ]*\d+,?\d+[ ]*[k€$]+', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(cdi|CDI|Cdi|cdd|CDD|Cdd|stage[s]*|stagiaire[s]*|intern[s]*|internship[s]*|freelance|Freelance|FREELANCE||indépendant[s]*|par an|/an|temps plein|3dexperience)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(h[/ \-]*f|f[/ \-]*h)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(salaire|[\d+]+[ ]*an[nées]*|[\d+]+[ ]*jour[s]|/mois|/jour[s]*|/semaine[s]*|[\d+]+ mois|[\d+]+ semaine[s]*|mois|semaine[s]*|jour[s]*])', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '[\d+]+[]*[èe]*[mes]*', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = r'\d+', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '[\w\.-]+@[\w\.-]+', repl = '', string = string, flags=re.IGNORECASE)
        return string

#On tokenize notre texte, on retire les pronoms, stop words, et on réduit chaque terme à sa racine
def tokenize(string):
    parsed_string = parser(string)
    tokenized_string = [word.lemma_.lower().strip() if word.lemma_ != '-PRON-' else word.lower_ for word in parsed_string]
    cleaned_tokenized_string = [word for word in tokenized_string if word not in punctuations and word not in stop_words]
    return cleaned_tokenized_string

#On retire la ponctuaction indésirable qui peut rester dans nos tokens
def strip_final_punctuation(list_of_tokens):
    forbid_car= [":", ";", ",", "&", "(", ")",
                '"', "!", "?", "*", "-", 
                "...", "/","'", "\\", "·", ".", "∙", "•"
                "–"]
    
    for i in range(len(list_of_tokens)):
        for car in forbid_car:
            list_of_tokens[i] = list_of_tokens[i].replace(car, '')
    return list_of_tokens
    


On définit ensuite une fonction finale de nettoyage qui tournera sur nos descriptions et sur nos titres.

In [194]:
def cleaning(string):
    cleaned_string = clean_and_lower(string)
    cleaned_string = terme_a_retirer(cleaned_string)
    tokenized_string = tokenize(cleaned_string)
    tokenized_string = strip_final_punctuation(tokenized_string)
    return tokenized_string
    

## Traitement des descriptions avec SpaCy

### I. Tokenize, clean, lemmatize

In [222]:
#On tokenize complètement avec spaCy nos descriptions

job_to_class['spacy_description'] = [cleaning(job_to_class.loc[i, 'texte']) for i in range(len(job_to_class['texte']))]

C:\Users\David\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### II. Create the TF-IDF term-documents matrix with the processed data

Il nous faut maintenant évaluer le poid relatif de chaque terme dans nos descriptions. On choisit donc de vectoriser en utilisant une matrice Tf-Idf.

In [216]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

L'initialisation des paramètres de notre vectorisation reste encore une question ouverte. L'idéal aurait été de tester automatiquement plusieurs nombre maximal de features et plusieurs n_gram, les stocker dans un log, et choisir le plus adapté. Faute de temps, on a choisi un grand nombre de features.

In [223]:
#On initialise nos paramètres pour la vectorisation

max_features_description = 5000
min_n_description = 1
max_n_description = 3

#On crée une fonction inutile. En effet TfidfVectorizer ne peux pas gérer des documents déjà tokenisés

def dummy_fun(doc): 
    return doc

#On vectorise

tfidf_vectorizer_description = TfidfVectorizer( analyzer = 'word',
                                    tokenizer = dummy_fun,
                                    preprocessor = dummy_fun,
                                    token_pattern = None,
                                    max_df = 1.0, min_df = 1,
                                    max_features = max_features_description, sublinear_tf = True,
                                    ngram_range=(min_n_description, max_n_description))


tfidf_matrix_description = tfidf_vectorizer_description.fit_transform(job_to_class['spacy_description'])


### III. Get the features

On utilise maintenant notre matrice term-document pour extraire les features les plus importantes de nos descriptions. C'est celles-ci qui vont ensuite nous servir pour l'extraction de topic.

In [224]:
features_description = tfidf_vectorizer_description.get_feature_names()

In [225]:
print(features_description)

['', ' profil', ' yu', ' yu continuer', '7e', '7e externe', 'ability', 'ability work', 'able', 'aborder', 'accenture', 'accepter', 'access', 'accessible', 'accompagnement', 'accompagnement client', 'accompagnement client dan', 'accompagner', 'accompagner client', 'accompagner client dan', 'accompagner croissance', 'accompagner dan', 'accompagner dan évolution', 'accompagner développement', 'accompagner entreprise', 'accompagner quotidien', 'accompagner transformation', 'accompagner équiper', 'accompagner évolution', 'accomplir', 'accomplish', 'accomplish extraordinary', 'accomplish extraordinary ground', 'accord', 'according', 'account', 'account manager', 'accounting', 'accounts', 'accroître', 'accueil', 'accueillir', 'accès', 'accélération', 'accélérer', 'accélérer cgi', 'accélérer cgi plan', 'achat', 'achats', 'achieve', 'achieve excellence', 'achieve excellence diversity', 'achieving', 'achieving workforce', 'achieving workforce diversity', 'acquis', 'acquisition', 'acquérir', 'act

### IV. Find the (features x topics) matric with the LDA/LSI/NMF and generate groups


Une fois nos features extraites, on choisit d'utiliser trois méthodes pour extraire les topics: Nonnegative Matrix Factorisation, Latent Dirichlet Allocation et Latent Semantic Index.
On créé nos trois matrices d'extraction.

In [226]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD

In [256]:
lda_description = LatentDirichletAllocation(n_components= 10, max_iter=10, learning_method='online',verbose=True)
data_lda_description = lda_description.fit_transform(tfidf_matrix_description)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [227]:
# Non-Negative Matrix Factorization Model
nmf_description = NMF(n_components = 10)
data_nmf_description = nmf_description.fit_transform(tfidf_matrix_description)

In [258]:
# Latent Semantic Indexing Model using Truncated SVD
lsi_description = TruncatedSVD(n_components = 10)
data_lsi_description = lsi_description.fit_transform(tfidf_matrix_description)

On imprime nos divers topics pour observer s'il y a de la cohérence et si c'est exploitable.

In [228]:
# Functions for printing keywords for each topic
def selected_topics(model, top_n = 10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(tfidf_vectorizer_description.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [229]:
print("LDA_description Model:")
selected_topics(lda_description)

In [230]:
print("LSI_description Model:")
selected_topics(lsi_description)

In [231]:
print("NMF_description Model:")
selected_topics(nmf_description)

NMF_description Model:
Topic 0:
[('développement', 0.7514516266294466), ('web', 0.6699034298620286), ('équiper', 0.6319129176587012), ('technique', 0.605646167150758), ('agile', 0.5437631600931951), ('application', 0.5317627194075365), ('javascript', 0.5309480171930088), ('coder', 0.49742467415290004), ('php', 0.46778589331301834), ('travailler', 0.4672692658843667)]
Topic 1:
[('team', 0.8486810110414359), ('experience', 0.7704660245704693), ('work', 0.6441642254508438), ('skills', 0.6134103282020048), ('haver', 0.574542001151327), ('new', 0.49924788674700127), ('dater', 0.4808332052680523), ('strong', 0.46294242655080375), ('business', 0.45385224724828543), ('development', 0.44998175474653296)]
Topic 2:
[('activus', 0.37799650959943554), ('proximité', 0.29563854462933625), ('savoir accompagner', 0.28384961842356127), ('recruter', 0.27820458871048237), ('consultant', 0.2574900176943783), ('perspective', 0.2571751815599714), ('recruter consultant', 0.25178937886923275), ('renforcer', 0.

## Traitement des titres avec SpaCy

Ce qui a été obtenu via le traitement des descriptions n'est finalement pas satisfaisant. On essaye ici de concentrer le traitement sur les titres de chaque annonce. L'intuition c'est que l'information pour clusteriser nos annonces est plus concentrée sur les titres. Et donc la possibilité de différenciation sera plus importante.

Les étapes étant les mêmes que pour la description, on ne répétera pas les commentaires précédents. On se contentera d'analyser les résultats.

### I. Tokenize, clean, lemmatize

In [195]:
job_to_class['spacy_titre'] = [cleaning(job_to_class.loc[i, 'titre']) for i in range(len(job_to_class['titre']))]

C:\Users\David\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### II. Create the TF-IDF term-documents matrix with the processed data

In [142]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Ici on a fait le choix de réduire les max_features à 500, et le n_gram à 1. En effet, les titres sont bien plus court, et bien plus redondants. Mais là aussi il aurait fallu tester automatiquement plusieurs valeurs et sélectionner celles qui donne les meilleures métriques.

In [212]:
#On initialise nos paramètres pour la vectorisation

max_features_titres = 500
min_n_titres = 1
max_n_titres = 1

#On crée une fonction inutile. En effet TfidfVectorizer ne peux pas gérer des documents déjà tokenisés

def dummy_fun(doc): 
    return doc

#On vectorise

tfidf_vectorizer_titres = TfidfVectorizer( analyzer = 'word',
                                    tokenizer = dummy_fun,
                                    preprocessor = dummy_fun,
                                    token_pattern = None,
                                    max_df = 1.0, min_df = 1,
                                    max_features = max_features_titres, sublinear_tf = True,
                                    ngram_range=(min_n_titres, max_n_titres))


tfidf_matrix_titres = tfidf_vectorizer_titres.fit_transform(job_to_class['spacy_titre'])


### III. Get the features !

In [213]:
features_titres = tfidf_vectorizer_titres.get_feature_names()
features_titres

['',
 '7e',
 '@pwc',
 'abap',
 'account',
 'achats',
 'acquisition',
 'actuaire',
 'administrateur',
 'administrator',
 'advisory',
 'affairer',
 'affaires',
 'agence',
 'agencer',
 'agile',
 'al',
 'alternance',
 'alternant',
 'alternante',
 'amoa',
 'analyse',
 'analyst',
 'analyste',
 'analytics',
 'android',
 'anglais',
 'angular',
 'angularjs',
 'api',
 'applicatif',
 'application',
 'applications',
 'apprentissage',
 'architect',
 'architecte',
 'architecture',
 'artificielle',
 'aspnet',
 'assistant',
 'assistante',
 'associate',
 'assurance',
 'ational',
 'audit',
 'auditeur',
 'automation',
 'automatisation',
 'automobile',
 'avantvente',
 'aws',
 'axa',
 'azure',
 'aéronautique',
 'b',
 'bancaire',
 'banque',
 'base',
 'based',
 'baser',
 'bb',
 'bi',
 'big',
 'bigdata',
 'btob',
 'business',
 'bw',
 'c',
 'c#net',
 'campaign',
 'center',
 'centre',
 'cfo',
 'chain',
 'change',
 'chantier',
 'charge',
 'charger',
 'chargé',
 'chargée',
 'chef',
 'chief',
 'client',
 'clients'

### IV. Find the (features x topics) matric with the LDA/LSI/NMF and generate groups

In [182]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD

In [183]:
lda_titres = LatentDirichletAllocation(n_components= 20, max_iter=10, learning_method='online',verbose=True)
data_lda_titres = lda_titres.fit_transform(tfidf_matrix_titres)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [357]:
df_topic_20 = pd.DataFrame(data_lda_titres)
df_topic_20
df_topic_20.to_csv("topic_20.csv")

In [219]:
# Non-Negative Matrix Factorization Model
nmf_titres = NMF(n_components = 20)
data_nmf_titres = nmf_titres.fit_transform(tfidf_matrix_titres)

In [187]:
# Latent Semantic Indexing Model using Truncated SVD
lsi_titres = TruncatedSVD(n_components = 15)
data_lsi_titres = lsi_titres.fit_transform(tfidf_matrix_titres)

In [204]:
# Functions for printing keywords for each topic
def selected_topics(model, top_n = 15):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(tfidf_vectorizer_titres.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [208]:
# print("LDA_titres Model:")
# type(selected_topics(lda_titres))

In [209]:
# print("LSI_titres Model:")
# selected_topics(lsi_titres)

In [220]:
print("NMF_titres Model:")
print(type(selected_topics(nmf_titres)))

NMF_titres Model:
Topic 0:
[('développeur', 6.607743426447681), ('python', 0.47146232538874716), ('informatique', 0.27540543664176687), ('android', 0.24001887597081448), ('concepteur', 0.21767208377440908), ('angular', 0.17951855578926051), ('logiciel', 0.15387793500966132), ('react', 0.15319435458367406), ('ios', 0.1502756565441051), ('mobile', 0.14563296980997023), ('analyste', 0.13976105961376137), ('javascript', 0.12480775094211563), ('sql', 0.12240896804006231), ('kend', 0.11457165560364041), ('salesforce', 0.10446063022250011)]
Topic 1:
[('data', 5.026926171198431), ('scientist', 2.3823911269767124), ('big', 1.1641682131497664), ('science', 0.2126789084393263), ('lead', 0.15741633942180944), ('architect', 0.13626924367843748), ('–', 0.12403698528564958), ('analyste', 0.10325436444980121), ('analytics', 0.09859649758107236), ('tech', 0.08167653495857265), ('ingenieur', 0.06105126962463851), ('consultante', 0.060809218191784795), ('protection', 0.04603180272909367), ('intelligence'

[('développement', 3.7099226173944455), ('d', 1.350199542920031), ('responsable', 1.0690385600441057), ('étude', 0.9857774863932953), ('logiciel', 0.6131697293941284), ('informatique', 0.4404888456345535), ('python', 0.3935454030128623), ('etudes', 0.36127866728636915), ('commercial', 0.3393525292114552), ('chargé', 0.28519828353064225), ('–', 0.2304376219048842), ('système', 0.19734713620808073), ('application', 0.1953694588861208), ('etude', 0.18318932244944464), ('affairer', 0.18006398646235325)]
Topic 17:
[('net', 3.9408727798697147), ('lead', 0.4177167829975332), ('core', 0.17441436571952815), ('tech', 0.12095814845653903), ('azure', 0.1153934401930175), ('angular', 0.08064404323560315), ('développeurs', 0.06275732817945033), ('wpf', 0.04792021153091603), ('développeur', 0.041443496158863995), ('expert', 0.040347671817452746), ('editeur', 0.039430616759255833), ('sharepoint', 0.033873749429326416), ('developer', 0.03216938551850057), ('technique', 0.03149021349635194), ('sénior', 

In [221]:
df_topic_20 = pd.DataFrame(data_nmf_titres)
df_topic_20.to_csv("topic_20.csv")

In [233]:
df_topic_20.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
count,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000
mean,0.014234,0.008495,0.007349,0.007135,0.008022,0.006779,0.008153,0.005167,0.003239,0.002750,0.005923,0.004774,0.005947,0.004171,0.006431,0.004675,0.007009,0.004796,0.004365,0.006066
std,0.027063,0.026370,0.032176,0.025587,0.030198,0.029135,0.028388,0.025032,0.015333,0.016313,0.026082,0.024399,0.023883,0.021561,0.026627,0.025976,0.021943,0.026361,0.022411,0.028576
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.026501,0.000168,0.000163,0.000937,0.000048,0.000794,0.000887,0.000455,0.000047,0.000014,0.000583,0.000084,0.000519,0.000000,0.000262,0.000010,0.002600,0.000059,0.000000,0.000381
max,0.148911,0.164515,0.207756,0.188958,0.213124,0.220277,0.211917,0.214926,0.134359,0.148693,0.213726,0.205111,0.201712,0.173640,0.222476,0.247675,0.192563,0.249517,0.217236,0.267637


## Conclusion

L'application de notre méthode de NLP sur les titres s'avère donner des extractions de topic plus cohérent que celle sur les descriptions. Toutefois l'ajout de notre matrice NMF dans les features de notre dataset nous fait gagner une accuracy relativement marginale. Et malheureusement les topics ne sont pas suffisamment cohérent pour alimenter la data analysis côté client.
Peut-être que la libraire Bert pourrait s'avérer plus utile sur ces deux niveaux.